## 1. Prepare Your Custom Dataset

In [ ]:
!pip install datasets

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import pandas as pd
from datasets import Dataset

#loading file through upload button
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Load the dataset
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    return df

# Prepare the dataset for GPT-2
def prepare_dataset(df, tokenizer):
    # Concatenate question and response as a single text
    df['text'] = df['question'] + tokenizer.eos_token + df['response']
    # Tokenize the text
    tokenized_data = df['text'].apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=128))
    # Convert to Dataset object from the datasets library
    return Dataset.from_pandas(pd.DataFrame(tokenized_data.tolist()))

# Load the dataset
file_path = "chatbot-dialogs1.csv"
df = load_dataset(file_path)

Saving chatbot-dialogs1.csv to chatbot-dialogs1 (2).csv
User uploaded file "chatbot-dialogs1 (2).csv" with length 250722 bytes


## 2. Fine-Tune the Model

In [ ]:
import transformers
import torch
import accelerate

print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)
print("Accelerate version:", accelerate.__version__)


Transformers version: 4.42.4
Torch version: 2.3.1+cu121
Accelerate version: 0.32.1


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Padding token
tokenizer.pad_token = tokenizer.eos_token

# Sample a fraction of the dataset for quick testing
sample_df = df.sample(frac=0.1, random_state=42)  # 10% of the data

# Prepare the dataset
train_dataset = prepare_dataset(df, tokenizer)

# Define the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1, # number of epochs
    per_device_train_batch_size=2,  # Adjust batch size if needed1
    save_steps=10_000,  # Save model more frequently
    save_total_limit=2,
    prediction_loss_only=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Step,Training Loss
500,3.222200
1000,2.950300
1500,2.755200


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

## 3. Predict with the Fine-Tuned Model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

# Function to predict the response
def predict_response(question, model, tokenizer, max_length=50, temperature=0.7, top_k=50, top_p=0.9, repetition_penalty=1.2):
    input_ids = tokenizer.encode(question, return_tensors='pt')
    outputs = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,       # Controls randomness: lower is more deterministic, higher is more random
        top_k=top_k,                   # Limits sampling to the top k tokens
        top_p=top_p,                   # Nucleus sampling: selects tokens with cumulative probability up to p
        repetition_penalty=repetition_penalty,  # Applies a penalty to repeated tokens
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Chat with the bot
print("Start chatting with the bot (type 'quit' to stop)!")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = predict_response(user_input, fine_tuned_model, fine_tuned_tokenizer)
    print(f"Bot: {response}")


Start chatting with the bot (type 'quit' to stop)!
You: hi
Bot: hi, i'm sorry. it's too late. we can't afford it. let me borrow a pair of shoes for you. i'll be glad to lend you one. i don? i think so. what do you want to buy?
You: what school do you go to?
Bot: what school do you go to? i'm a math major. what's the matter with that? it's not my first year. how many years have you been in high school, anyway.? i've only started out at middle school. i don
You: hi, how are you doing?
Bot: hi, how are you doing? i'm fine. i'll be there when you're ready. i hope you don't mind. i've been busy lately. what's the matter with that? i haven' t had a good day. i
You: quit


## (Optional) Generate Text with the Fine-Tuned Model

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

# Function to generate text with attention mask and improved diversity
def generate_text(prompt, model, tokenizer, max_length=50, temperature=0.7, top_k=50, top_p=0.9):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = torch.ones(inputs.shape, dtype=torch.long)

    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        temperature=temperature,  # Controls randomness: lower is more deterministic, higher is more random
        top_k=top_k,              # Limits sampling to the top k tokens
        top_p=top_p,              # Nucleus sampling: selects tokens with cumulative probability up to p
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Your custom prompt here"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer)
print(f"Generated text: {generated_text}")


E:\BIT\Project 8th(Chatbot)\Code\ChatBot\chatbotEnv\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
E:\BIT\Project 8th(Chatbot)\Code\ChatBot\chatbotEnv\Lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated text: Your custom prompt here?
i'm not sure.	i'm not sure.
i'm not sure.	i'm not sure.
i'm not sure.	what's the matter with that?
what's the matter with that
